In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sorted_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from scipy import interpolate
from frame_count import frame_count
from scipy.signal import savgol_filter
from scipy.spatial import distance


In [ ]:
path = '/Users/andrei-macpro/Documents/Data/pose/meal_openpose/1043_meal'
detects = sorted_alpha(path)
numbers = [int((x.split("."))[0].split("_")[1]) for x in detects]

In [ ]:
neck_keypoints

In [ ]:
import pandas as pd
import numpy as np

def process_skeleton_data(csv_file_path):
    # 1. read data
    data = pd.read_csv(csv_file_path)
    temp_data = data.copy()

    
    # 2. drop skeletons with missing neck
    for i in range(0, len(data.columns[1:]), 3):
        neck_value = data[data.columns[1:][i]].loc[1]
        if pd.isna(neck_value) or neck_value == 0:
            temp_data = temp_data.drop([data.columns[1:][i], data.columns[1:][i+1], data.columns[1:][i+2]], axis=1)  # Drop skeletons with missing neck
    data = temp_data

    # 3. if no skeletons with neck return None
    if len(data.columns) < 4:  # If not enough skeletons with neck present, return None
        return None, None

    # remove fourth skeleton if detected
    elif len(data.columns) > 10:
        # Drop the fourth skeleton if it has been detected
        data = data.drop(data.columns[10:], axis=1) 

    # 4. initialize the tracking with the first two skeletons and save the neck_points
    detections = []
    neck_keypoints = []
    for col in range(0, len(data.columns[1:]), 3):
        skeleton = data.iloc[:, col:col+3].values
        detections.append(skeleton[:,1:])  # Select only x and y columns
        neck_keypoints.append(skeleton[:,1:][1])  # Assuming neck keypoint is the second row (index 1)
    
    detections = np.array(detections)  # Shape: (num_skeletons, num_keypoints, 2)
    neck_keypoints = np.array(neck_keypoints)  # Shape: (num_skeletons, 2)

    return detections, neck_keypoints

# Example usage
csv_file_path = '/Users/andrei-macpro/Documents/Data/pose/meal_openpose/1043_meal/frame_796.csv'
detections, neck_keypoints = process_skeleton_data(csv_file_path)

print(neck_keypoints)

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

# Initialize tracking variables
tracks = []
frames = []
knn = NearestNeighbors(n_neighbors=1, metric='euclidean')

# Example usage
path = '/Users/andrei-macpro/Documents/Data/pose/meal_openpose/1043_meal'
detects = sorted_alpha(path)
numbers = [int(f.split('_')[-1].split('.')[0]) for f in detects]

for i, number in zip(range(len(detects)), numbers):
    frames.append(number)
    print('frame no:', number)
    
    data = pd.read_csv(os.path.join(path, detects[i]))
    
    if i == 0:  # For the first frame, just get the skeletons, don't do anything
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))
        # Initialize track lists for each detection
        for _ in range(len(detections)):
            tracks.append([])

        # Append each detection to its corresponding track list
        for j, detection in enumerate(detections):
            tracks[j].append(detection)

    else:
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))
        # Get the last neck positions from the tracks
        last_necks = np.array([track[-1][1] for track in tracks])

        # Fit the KNN model on the last neck positions
        knn.fit(last_necks)
        # Keep track of which detections have been assigned
        assigned_detections = set()

        # Store distances and indices
        distances_indices = []
        # Compute Euclidean distances between each pair of new detections 
        necks_array = np.array(necks)
        num_detections = len(necks_array)
        distances = np.zeros((num_detections, num_detections))


        for j in range(num_detections):
            for k in range(j + 1, num_detections):
                dist = euclidean(necks_array[j], necks_array[k])
                distances[j, k] = dist
                distances[k, j] = dist  # Symmetric matrix
        # Check if distance between the first 2 detections are more than 200
        if num_detections > 1:
            if i <50:
                if  np.any(distances[0:2,0:2]) > 200:
                    # Find the closest track for each new detection
                    for j, neck in enumerate(necks):
                        dist, index = knn.kneighbors([neck])
                        distances_indices.append((dist[0][0], index[0][0], j))

                    # Sort detections by distance to the closest track
                    distances_indices.sort()

                    # Assign the closest detections to the tracks
                    for dist, closest_track_index, j in distances_indices[:len(tracks)]:
                        tracks[closest_track_index].append(detections[j])
                        assigned_detections.add(j)

                    # If there are more detections than tracks, create new tracks for the remaining detections
                    for dist, closest_track_index, j in distances_indices[len(tracks):]:
                        new_track_index = len(tracks)
                        tracks.append([])  # Initialize a new track
                        tracks[new_track_index].append(detections[j])
                        assigned_detections.add(j)
                else:
                    relative_position = relative_position(tracks[0], tracks[1], necks[0], necks[1])
                    if relative_position == 1:
                        tracks[0].append(detections[0])
                        tracks[1].append(detections[1])
                    else:
                        tracks[0].append(detections[1])
                        tracks[1].append(detections[0])
                    
        elif num_detections == 1:
            if np.any(distances) > 200:
            # this should still do the relative position calculation if distance<200 
                for j, neck in enumerate(necks):
                    dist, index = knn.kneighbors([neck])
                    tracks[index[0][0]].append(detections[j])
                    assigned_detections.add(j)
            else:
                if i <50:
                    for j, neck in enumerate(necks):
                        dist, index = knn.kneighbors([neck])
                        tracks[index[0][0]].append(detections[j])
                        assigned_detections.add(j)

                   ## think about this : what if there's only one detection and i>50?  
                else:
                    relative_position = relative_position(tracks[0], tracks[1], necks[0], necks[1])
                    if relative_position == 1:
                        tracks[0].append(detections[0])
                        tracks[1].append(detections[1])
                    else:
                        tracks[0].append(detections[1])
                        tracks[1].append(detections[0])



### the case where num_detections > 1 and distances <200  and i < 50 is not handled yet


           




In [ ]:
def relative_position(tracks, necks):
    num_tracks = len(tracks)
    num_necks = len(necks)

    if num_necks == 1:
        neck = necks[0]
        track1, track2 = tracks[:2]

        x_diff = [d1[0][0] - d2[0][0] for d1, d2 in zip(track1[-50:], track2[-50:])]
        y_diff = [d1[0][1] - d2[0][1] for d1, d2 in zip(track1[-50:], track2[-50:])]

        mean_x_diff = np.mean(x_diff)
        mean_y_diff = np.mean(y_diff)

        if abs(mean_x_diff) > abs(mean_y_diff):
            return 1 if neck[0] > (track1[-1][0][0] + track2[-1][0][0]) / 2 else 2
        else:
            return 1 if neck[1] > (track1[-1][0][1] + track2[-1][0][1]) / 2 else 2

    track1, track2 = tracks[:2]
    neck1, neck2 = necks[:2]

    x_diff = [d1[0][0] - d2[0][0] for d1, d2 in zip(track1[-50:], track2[-50:])]
    y_diff = [d1[0][1] - d2[0][1] for d1, d2 in zip(track1[-50:], track2[-50:])]

    mean_x_diff = np.mean(x_diff)
    mean_y_diff = np.mean(y_diff)

    if abs(mean_x_diff) > abs(mean_y_diff):  # is x axis difference greater than y axis difference
        if mean_x_diff > 0:  # what's their relative position on x axis
            return 1 if neck1[0] > neck2[0] else 2  # maintain relative position
        else:
            return 1 if neck1[0] < neck2[0] else 2
    else:  # is y axis difference greater than x axis difference
        if mean_y_diff > 0:  # what's their relative position on y axis
            return 1 if neck1[1] > neck2[1] else 2  # maintain relative position
        else:
            return 1 if neck1[1] < neck2[1] else 2

In [ ]:
from scipy.spatial.distance import euclidean
import os
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.optimize import linear_sum_assignment

# Initialize tracking variables
tracks = []
frames = []
knn = NearestNeighbors(n_neighbors=1, metric='euclidean')
threshold = 80
# Example usage
path = '/Users/andrei-macpro/Documents/Data/pose/meal_openpose/1043_meal'
detects = sorted_alpha(path)
numbers = [int(f.split('_')[-1].split('.')[0]) for f in detects]


for i, number in zip(range(len(detects)), numbers):
    frames.append(number)
    print('frame no:', number)
    
    data = pd.read_csv(os.path.join(path, detects[i]))
    
    if i == 0:  # For the first frame, just get the skeletons, don't do anything
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))
        # Initialize track lists for each detection
        for _ in range(len(detections)):
            tracks.append([])

        # Append each detection to its corresponding track list with the frame number
        for j, detection in enumerate(detections):
            tracks[j].append((detection, number))

    else:
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))
        
        # Check if there are valid neck joints
        if necks is None or len(necks) == 0 or all(neck is None for neck in necks):
            continue
        
        # Get the last neck positions from the tracks
        last_necks = np.array([track[-1][0][1] for track in tracks])

        # Fit the KNN model on the last neck positions
        knn.fit(last_necks)
        # Keep track of which detections have been assigned
        assigned_detections = set()
        # Store distances and indices
        distances_indices = []
        # Compute Euclidean distances between each pair of new detections 
        necks_array = np.array(necks)
        num_detections = len(necks_array)
        distances = np.zeros((num_detections, num_detections))

        for j in range(num_detections):
            for k in range(j + 1, num_detections):
                dist = euclidean(necks_array[j], necks_array[k])
                distances[j, k] = dist
                distances[k, j] = dist  # Symmetric matrix

        # Handle the case where there is only one detection
        if num_detections == 1:
            if len(tracks) == 1:
                tracks[0].append((detections[0], number))
            else:
                # Assign the single detection to the closest track
                dist, index = knn.kneighbors([necks[0]])
                tracks[index[0][0]].append((detections[0], number))
        else:
            # Check if distance between the first 2 detections is less than 80
            if i > 50 and np.any(distances[0,1] < threshold):
                if len(tracks) == 1:
                    for j, neck in enumerate(necks):
                        dist, index = knn.kneighbors([neck])
                        distances_indices.append((dist[0][0], index[0][0], j))
                        # Sort detections by distance to the closest track
                        distances_indices.sort()
                        # Assign the closest detections to the tracks
                        for dist, closest_track_index, j in distances_indices[:len(tracks)]:
                            tracks[closest_track_index].append((detections[j], number))
                            assigned_detections.add(j)
                else:
                    relative_pos = relative_position(tracks, necks)
                    if relative_pos == 1:
                        tracks[0].append((detections[0], number))
                        tracks[1].append((detections[1], number))
                    else:
                        tracks[0].append((detections[1], number))
                        tracks[1].append((detections[0], number))
            else:
                # Find the closest track for each new detection
                for j, neck in enumerate(necks):
                    dist, index = knn.kneighbors([neck])
                    distances_indices.append((dist[0][0], index[0][0], j))


                # Sort detections by distance to the closest track
                distances_indices.sort()
                if len(detections) == 3 and len(tracks) == 2:
                     # Assign the closest detections to the tracks using k-NN
                    assigned_detections = set()
                    for dist, closest_track_index, j in distances_indices[:2]:
                        tracks[closest_track_index].append((detections[j], number))
                        assigned_detections.add(j)

          
       
                else:

                    # Assign the closest detections to the tracks
                    for dist, closest_track_index, j in distances_indices[:len(tracks)]:
                        tracks[closest_track_index].append((detections[j], number))
                        assigned_detections.add(j)

                 

In [ ]:
len(tracks)

In [ ]:
import pandas as pd

# Example structure of a detection (assuming each detection is a list of keypoints, each keypoint is a list [x, y])
# detections = [[[x1, y1], [x2, y2], ...], [[x1, y1], [x2, y2], ...], ...]

# Flatten the tracks list and extract the detection, frame number, and track index
flattened_tracks = [(detection, frame, track_index) for track_index, track in enumerate(tracks) for detection, frame in track]

# Initialize a list to store the data for the DataFrame
data = []

# Iterate over the flattened tracks to extract keypoints
for detection, frame, track_index in flattened_tracks:
    for keypoint_index, (x, y) in enumerate(detection):
        data.append({'Frame': frame, 'Track': track_index, 'Keypoint': keypoint_index, 'X': x, 'Y': y})

# Create a DataFrame from the data
df_tracks = pd.DataFrame(data)

# Set the frame number, keypoint, and track as a multi-index
df_tracks.set_index(['Frame', 'Keypoint', 'Track'], inplace=True)

# Display the DataFrame
print(df_tracks)

In [ ]:
import pandas as pd

# Example structure of a detection (assuming each detection is a list of keypoints, each keypoint is a list [x, y])
# detections = [[[x1, y1], [x2, y2], ...], [[x1, y1], [x2, y2], ...], ...]

# Flatten the tracks list and extract the detection, frame number, and track index
flattened_tracks = [(detection, frame, track_index) for track_index, track in enumerate(tracks) for detection, frame in track]

# Initialize a list to store the data for the DataFrame
data = []

# Iterate over the flattened tracks to extract keypoints
for detection, frame, track_index in flattened_tracks:
    for keypoint_index, (x, y) in enumerate(detection):
        data.append({'Frame': frame, 'Track': track_index, 'Keypoint': keypoint_index, 'X': x, 'Y': y})

# Create a DataFrame from the data
df_tracks = pd.DataFrame(data)

# Pivot the DataFrame to have 'Track' as columns and keypoints as rows
df_pivot = df_tracks.pivot_table(index=['Frame', 'Keypoint'], columns='Track', values=['X', 'Y'])

# Flatten the multi-level columns
df_pivot.columns = [f'{coord}_Track{track}' for coord, track in df_pivot.columns]

# Reset the index to make 'Frame' and 'Keypoint' columns
df_pivot.reset_index(inplace=True)

# Display the DataFrame
print(df_pivot)

## now draw bounding boxes before imputation 

In [ ]:
video_path = '/Users/andrei-macpro/Documents/Data/videos/meal_videos/1043_meal.mp4'
cap = cv2.VideoCapture(video_path)
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]
# Get the width and height of the frames
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

output_path = '/Users/andrei-macpro/Documents/Data/videos/1043_meal_annotated.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Flag to toggle frame number display
show_frame_number = True

# Iterate over the video frames
frame_count = 1
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get the rows for the current frame
    frame_data = df_pivot[df_pivot['Frame'] == frame_count]

    # Iterate over each track
    for track in range(len(tracks)):
        # Get the keypoints for the current track
        keypoints = frame_data[[f'X_Track{track}', f'Y_Track{track}']].values

        # Choose a color for the current track
        color = colors[track % len(colors)]

        # Draw bounding boxes around the keypoints and display track number
        for x, y in keypoints:
            if not pd.isna(x) and not pd.isna(y):
                cv2.rectangle(frame, (int(x) - 5, int(y) - 5), (int(x) + 5, int(y) + 5), color, 2)
                cv2.putText(frame, f'Track {track}', (int(x) + 10, int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    # Display the frame number if the flag is set
    if show_frame_number:
        cv2.putText(frame, f'Frame {frame_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)



    # Write the frame to the output video
    out.write(frame)

    frame_count += 1

# Release the video capture object and close all OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# save df_pivot to a csv file
df_pivot.to_csv('output.csv')